In [142]:
'''Imports'''

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as MSE




In [76]:
'''load and visualize train dataset'''

train_df = pd.read_csv('Datasets/SE_rents2018_train.csv')
test_df = pd.read_csv('Datasets/SE_rents2018_test1.csv')


In [71]:
'''checking features with highest correlations to rent'''

train_df.corr()[['rent']].sort_values(by='rent',ascending=False)



,rent
rent,1.000000
size_sqft,0.649984
bathrooms,0.611987
bedrooms,0.353337
has_washer_dryer,0.343123
has_doorman,0.273486
floornumber,0.267151
has_concierge,0.260079
floor_count,0.253517
has_gym,0.248213


In [150]:
train_features = train_df[['size_sqft','bathrooms','bedrooms','has_washer_dryer']]
train_target = train_df.rent

test_features = test_df[['size_sqft','bathrooms','bedrooms','has_washer_dryer']]
test_target = test_df.rent

lreg = LinearRegression()

lreg.fit(train_features,train_target)

test_df['rent_pred'] = lreg.predict(test_features)

lreg.score(test_features,test_target)



0.44646273449617346

In [151]:
MSE(test_df.rent_pred,test_target)

4127818.5368494615

In [152]:
lreg.

array([   2.53908197, 1788.69664597, -160.3050474 ,  830.37632281])